## Task 1

In [1]:
import pandas as pd

loans = pd.read_csv('data/loans.csv')
loans

,id,account_id,date,amount,payments,24_A,12_B,12_A,60_D,48_C,...,60_C,24_B,48_D,24_D,48_B,36_A,36_B,60_B,12_D,60_A
0,4959,2,1994-01-05,80952,3373,X,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,4961,19,1996-04-29,30276,2523,-,X,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,4962,25,1997-12-08,30276,2523,-,-,X,-,-,...,-,-,-,-,-,-,-,-,-,-
3,4967,37,1998-10-14,318480,5308,-,-,-,X,-,...,-,-,-,-,-,-,-,-,-,-
4,4968,38,1998-04-19,110736,2307,-,-,-,-,X,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,7294,11327,1998-09-27,39168,1632,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
678,7295,11328,1998-07-18,280440,4674,-,-,-,-,-,...,X,-,-,-,-,-,-,-,-,-
679,7304,11349,1995-10-29,419880,6998,-,-,-,-,-,...,X,-,-,-,-,-,-,-,-,-
680,7305,11359,1996-08-06,54024,4502,-,-,X,-,-,...,-,-,-,-,-,-,-,-,-,-


Now examining the status fields and converting them to binary.

In [56]:
binary_status_fields = loans[status_fields].replace('X', 1).replace('-', 0)
binary_status_fields.sum(axis=1).describe()

count    682.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
dtype: float64

Seems like each row has a unique status as expected. Now melting the status fields and selecting when the value of the melted variable is 1.

In [43]:
loans2 = pd.concat([loans[['id', 'account_id', 'date', 'amount', 'payments']], binary_status_fields],axis=1)
melted = loans2.melt(id_vars=['id', 'account_id', 'date', 'amount', 'payments'])
loans3 = melted[melted['value'] == 1]
loans3

,id,account_id,date,amount,payments,variable,value
0,4959,2,1994-01-05,80952,3373,24_A,1
5,4973,67,1996-05-02,165960,6915,24_A,1
32,5088,544,1996-08-22,91152,3798,24_A,1
41,5131,816,1995-01-08,48624,2026,24_A,1
44,5136,846,1996-07-28,187104,7796,24_A,1
...,...,...,...,...,...,...,...
11946,6228,6034,1993-12-01,464520,7742,60_B,1
12759,6682,8225,1998-03-30,36204,3017,12_D,1
13498,6863,9188,1993-07-28,127080,2118,60_A,1
13572,7104,10320,1993-12-13,259740,4329,60_A,1


This has unpivoted the table and ensured that each row represents a unique customer. <br>
Now we are going to create three separate variables from the status field. These variables would indicate the loan term length in months, whether the loan is active or expired, and whether the payment is paid in full, default, or in progress.

In [55]:
import warnings
warnings.filterwarnings('ignore')
loans3['term'] = loans3['variable'].apply(lambda x: int(x.split("_")[0]))
loans3['status'] = loans3['variable'].apply(lambda x: x.split("_")[1])
loans3['active_status'] = loans3['status'].apply(lambda x: 'expired' if x in ['A','B'] else 'current')
loans3['payment'] = loans3['status'].apply(lambda x: 'full' if x == 'A' else ('in_progress' if x == 'C' else 'default'))
loans4 = loans3.drop(['variable', 'value','status'], axis=1)
loans4

,id,account_id,date,amount,payments,term,active_status,payment
0,4959,2,1994-01-05,80952,3373,24,expired,full
5,4973,67,1996-05-02,165960,6915,24,expired,full
32,5088,544,1996-08-22,91152,3798,24,expired,full
41,5131,816,1995-01-08,48624,2026,24,expired,full
44,5136,846,1996-07-28,187104,7796,24,expired,full
...,...,...,...,...,...,...,...,...
11946,6228,6034,1993-12-01,464520,7742,60,expired,default
12759,6682,8225,1998-03-30,36204,3017,12,current,default
13498,6863,9188,1993-07-28,127080,2118,60,expired,full
13572,7104,10320,1993-12-13,259740,4329,60,expired,full


## Task 2

In [101]:
districts = pd.read_csv('districts.csv')
districts

,id,name,region,population,num_cities,urban_ratio,avg_salary,entrepreneur_1000,municipality_info,unemployment_rate,commited_crimes
0,1,Hl.m. Praha,Prague,1204953,1,100.0,12541,167,"[0,0,0,1]","[0.2,0.43]","[85677,99107]"
1,2,Benesov,central Bohemia,88884,5,46.7,8507,132,"[80,26,6,2]","[1.6,1.85]","[2159,2674]"
2,3,Beroun,central Bohemia,75232,5,41.7,8980,111,"[55,26,4,1]","[1.9,2.21]","[2824,2813]"
3,4,Kladno,central Bohemia,149893,6,67.4,9753,109,"[63,29,6,2]","[4.6,5.05]","[5244,5892]"
4,5,Kolin,central Bohemia,95616,6,51.4,9307,118,"[65,30,4,1]","[3.8,4.43]","[2616,3040]"
...,...,...,...,...,...,...,...,...,...,...,...
72,73,Opava,north Moravia,182027,7,56.4,8746,90,"[17,49,12,2]","[3.3,3.74]","[4355,4433]"
73,74,Ostrava - mesto,north Moravia,323870,1,100.0,10673,100,"[0,0,0,1]","[4.7,5.44]","[18782,18347]"
74,75,Prerov,north Moravia,138032,5,64.6,8819,99,"[67,30,4,2]","[5.3,5.66]","[4063,4505]"
75,76,Sumperk,north Moravia,127369,7,51.2,8369,107,"[31,32,13,2]","[4.7,5.88]","[3736,2807]"


In [102]:
# Create 4 separate columns from municipality_info
districts['num_municipality_with_population_<500'] = districts['municipality_info'].apply(lambda x: int(x.split(",")[0][1:]))
districts['num_municipality_with_population_500-1999'] = districts['municipality_info'].apply(lambda x: int(x.split(",")[1]))
districts['num_municipality_with_population_2000-9999'] = districts['municipality_info'].apply(lambda x: int(x.split(",")[2]))
districts['num_municipality_with_population_>10000'] = districts['municipality_info'].apply(lambda x: int(x.split(",")[3][:-1]))
# Create 2 separate columns for unemployment rate
districts['unemployment_rate_95'] = districts['unemployment_rate'].apply(lambda x: x.split(",")[0][1:])
districts['unemployment_rate_96'] = districts['unemployment_rate'].apply(lambda x: x.split(",")[1][:-1])
# Create 2 separate columns for crime rate
districts['crime_rate_95'] = districts['commited_crimes'].apply(lambda x: x.split(",")[0][1:])
districts['crime_rate_96'] = districts['commited_crimes'].apply(lambda x: x.split(",")[1][:-1])
# Drop the municiplality info, unemployment rate and commited crimes
districts = districts.drop(['unemployment_rate', 'commited_crimes', 'municipality_info'],axis=1)
districts

,id,name,region,population,num_cities,urban_ratio,avg_salary,entrepreneur_1000,num_municipality_with_population_<500,num_municipality_with_population_500-1999,num_municipality_with_population_2000-9999,num_municipality_with_population_>10000,unemployment_rate_95,unemployment_rate_96,crime_rate_95,crime_rate_96
0,1,Hl.m. Praha,Prague,1204953,1,100.0,12541,167,0,0,0,1,0.2,0.43,85677,99107
1,2,Benesov,central Bohemia,88884,5,46.7,8507,132,80,26,6,2,1.6,1.85,2159,2674
2,3,Beroun,central Bohemia,75232,5,41.7,8980,111,55,26,4,1,1.9,2.21,2824,2813
3,4,Kladno,central Bohemia,149893,6,67.4,9753,109,63,29,6,2,4.6,5.05,5244,5892
4,5,Kolin,central Bohemia,95616,6,51.4,9307,118,65,30,4,1,3.8,4.43,2616,3040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,73,Opava,north Moravia,182027,7,56.4,8746,90,17,49,12,2,3.3,3.74,4355,4433
73,74,Ostrava - mesto,north Moravia,323870,1,100.0,10673,100,0,0,0,1,4.7,5.44,18782,18347
74,75,Prerov,north Moravia,138032,5,64.6,8819,99,67,30,4,2,5.3,5.66,4063,4505
75,76,Sumperk,north Moravia,127369,7,51.2,8369,107,31,32,13,2,4.7,5.88,3736,2807


In [106]:
districts.dtypes

id                                              int64
name                                           object
region                                         object
population                                      int64
num_cities                                      int64
urban_ratio                                   float64
avg_salary                                      int64
entrepreneur_1000                               int64
num_municipality_with_population_<500           int64
num_municipality_with_population_500-1999       int64
num_municipality_with_population_2000-9999      int64
num_municipality_with_population_>10000         int64
unemployment_rate_95                           object
unemployment_rate_96                           object
crime_rate_95                                  object
crime_rate_96                                  object
dtype: object

In [109]:
import numpy as np
districts['unemployment_rate_95'] = districts['unemployment_rate_95'].apply(lambda x: float(x) if x != 'NA' else np.nan)
districts['unemployment_rate_96'] = districts['unemployment_rate_96'].apply(lambda x: float(x) if x != 'NA' else np.nan)
districts['crime_rate_95'] = districts['crime_rate_95'].apply(lambda x: float(x) if x != 'NA' else np.nan)
districts['crime_rate_96'] = districts['crime_rate_96'].apply(lambda x: float(x) if x != 'NA' else np.nan)
districts.dtypes

id                                              int64
name                                           object
region                                         object
population                                      int64
num_cities                                      int64
urban_ratio                                   float64
avg_salary                                      int64
entrepreneur_1000                               int64
num_municipality_with_population_<500           int64
num_municipality_with_population_500-1999       int64
num_municipality_with_population_2000-9999      int64
num_municipality_with_population_>10000         int64
unemployment_rate_95                          float64
unemployment_rate_96                          float64
crime_rate_95                                 float64
crime_rate_96                                 float64
dtype: object

## Task 3

In [110]:
account = pd.read_csv('accounts.csv')
account

,id,district_id,date,statement_frequency
0,1,18,1995-03-24,monthly
1,2,1,1993-02-26,monthly
2,3,5,1997-07-07,monthly
3,4,12,1996-02-21,monthly
4,5,15,1997-05-30,monthly
...,...,...,...,...
4495,11027,32,1996-07-26,weekly
4496,11054,59,1996-02-01,weekly
4497,11138,70,1995-08-18,weekly
4498,11141,1,1997-02-11,weekly


In [116]:
combined1 = pd.merge(account, districts, left_on='district_id', right_on='id', how='left').drop(['id_y'], axis=1)
combined1.rename(columns={'id_x': 'account_id', 'name': 'district_name', 'date': 'open_date'}, inplace=True)
combined1

,account_id,district_id,open_date,statement_frequency,district_name,region,population,num_cities,urban_ratio,avg_salary,entrepreneur_1000,num_municipality_with_population_<500,num_municipality_with_population_500-1999,num_municipality_with_population_2000-9999,num_municipality_with_population_>10000,unemployment_rate_95,unemployment_rate_96,crime_rate_95,crime_rate_96
0,1,18,1995-03-24,monthly,Pisek,south Bohemia,70699,4,65.3,8968,131,60,13,2,1,2.8,3.35,1740.0,1910.0
1,2,1,1993-02-26,monthly,Hl.m. Praha,Prague,1204953,1,100.0,12541,167,0,0,0,1,0.2,0.43,85677.0,99107.0
2,3,5,1997-07-07,monthly,Kolin,central Bohemia,95616,6,51.4,9307,118,65,30,4,1,3.8,4.43,2616.0,3040.0
3,4,12,1996-02-21,monthly,Pribram,central Bohemia,107870,6,58.0,8754,137,84,29,6,1,3.8,4.31,3804.0,3868.0
4,5,15,1997-05-30,monthly,Cesky Krumlov,south Bohemia,58796,5,51.9,9045,124,22,16,7,1,3.1,3.60,1845.0,1879.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11027,32,1996-07-26,weekly,Ceska Lipa,north Bohemia,105058,10,81.0,9272,118,32,19,7,2,2.8,3.22,4340.0,4505.0
4496,11054,59,1996-02-01,weekly,Kromeriz,south Moravia,108871,6,62.1,8444,106,41,29,7,2,3.2,3.47,2595.0,2305.0
4497,11138,70,1995-08-18,weekly,Karvina,north Moravia,285387,7,89.9,10177,81,0,2,8,5,6.6,7.75,9878.0,10108.0
4498,11141,1,1997-02-11,weekly,Hl.m. Praha,Prague,1204953,1,100.0,12541,167,0,0,0,1,0.2,0.43,85677.0,99107.0


In [4]:
links = pd.read_csv('links.csv')
links

,id,client_id,account_id,type
0,1,1,1,owner
1,2,2,2,owner
2,3,3,2,user
3,4,4,3,owner
4,5,5,3,user
...,...,...,...,...
5364,13647,13955,11349,owner
5365,13648,13956,11349,user
5366,13660,13968,11359,owner
5367,13663,13971,11362,owner


In [3]:
cards = pd.read_csv('cards.csv')
cards

,id,link_id,type,issue_date
0,1,9,gold,1998-10-16
1,2,19,classic,1998-03-13
2,3,41,gold,1995-09-03
3,4,42,classic,1998-11-26
4,5,51,junior,1995-04-24
...,...,...,...,...
887,1230,13312,classic,1998-03-08
888,1233,13382,classic,1996-07-06
889,1234,13386,classic,1997-11-28
890,1239,13442,junior,1998-02-02


In [5]:
import pandas as pd
pd.merge(cards, links, left_on='link_id', right_on='id', how='right')

,id_x,link_id,type_x,issue_date,id_y,client_id,account_id,type_y
0,1.0,9.0,gold,1998-10-16,9,9,7,owner
1,2.0,19.0,classic,1998-03-13,19,19,14,owner
2,3.0,41.0,gold,1995-09-03,41,41,33,owner
3,4.0,42.0,classic,1998-11-26,42,42,34,owner
4,5.0,51.0,junior,1995-04-24,51,51,43,owner
...,...,...,...,...,...,...,...,...
5364,NaN,NaN,NaN,NaN,13623,13931,11333,owner
5365,NaN,NaN,NaN,NaN,13647,13955,11349,owner
5366,NaN,NaN,NaN,NaN,13648,13956,11349,user
5367,NaN,NaN,NaN,NaN,13663,13971,11362,owner


In [141]:
combined3 = pd.merge(cards, links, left_on='link_id', right_on='id', how='right')
combined3.rename(columns={'id_x': 'card_id', 'type_x': 'card_type', 'issue_date': 'card_issue_date', 'type_y': 'card_type'}, inplace=True)
combined3.drop(['id_y'], axis=1, inplace=True)
card_account = combined3.groupby(['account_id']).count().reset_index()
card_account


,account_id,card_id,link_id,card_type,card_issue_date,client_id,card_type
0,1,0,0,0,0,1,1
1,2,0,0,0,0,2,2
2,3,0,0,0,0,2,2
3,4,0,0,0,0,1,1
4,5,0,0,0,0,1,1
...,...,...,...,...,...,...,...
4495,11333,0,0,0,0,1,1
4496,11349,0,0,0,0,2,2
4497,11359,1,1,1,1,1,1
4498,11362,0,0,0,0,1,1


In [138]:
links_account = links.groupby(['account_id']).count().reset_index()
links_account.rename(columns={'client_id': 'num_customers'}, inplace=True)
links_account

,account_id,id,num_customers,type
0,1,1,1,1
1,2,2,2,2
2,3,2,2,2
3,4,1,1,1
4,5,1,1,1
...,...,...,...,...
4495,11333,1,1,1
4496,11349,2,2,2
4497,11359,1,1,1
4498,11362,1,1,1


In [131]:
combined2 = pd.merge(combined1, links_account, left_on='account_id', right_on='account_id', how='left')
combined2

,account_id,district_id,open_date,statement_frequency,district_name,region,population,num_cities,urban_ratio,avg_salary,...,num_municipality_with_population_500-1999,num_municipality_with_population_2000-9999,num_municipality_with_population_>10000,unemployment_rate_95,unemployment_rate_96,crime_rate_95,crime_rate_96,id,num_customers,type
0,1,18,1995-03-24,monthly,Pisek,south Bohemia,70699,4,65.3,8968,...,13,2,1,2.8,3.35,1740.0,1910.0,1,1,1
1,2,1,1993-02-26,monthly,Hl.m. Praha,Prague,1204953,1,100.0,12541,...,0,0,1,0.2,0.43,85677.0,99107.0,2,2,2
2,3,5,1997-07-07,monthly,Kolin,central Bohemia,95616,6,51.4,9307,...,30,4,1,3.8,4.43,2616.0,3040.0,2,2,2
3,4,12,1996-02-21,monthly,Pribram,central Bohemia,107870,6,58.0,8754,...,29,6,1,3.8,4.31,3804.0,3868.0,1,1,1
4,5,15,1997-05-30,monthly,Cesky Krumlov,south Bohemia,58796,5,51.9,9045,...,16,7,1,3.1,3.60,1845.0,1879.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11027,32,1996-07-26,weekly,Ceska Lipa,north Bohemia,105058,10,81.0,9272,...,19,7,2,2.8,3.22,4340.0,4505.0,1,1,1
4496,11054,59,1996-02-01,weekly,Kromeriz,south Moravia,108871,6,62.1,8444,...,29,7,2,3.2,3.47,2595.0,2305.0,1,1,1
4497,11138,70,1995-08-18,weekly,Karvina,north Moravia,285387,7,89.9,10177,...,2,8,5,6.6,7.75,9878.0,10108.0,1,1,1
4498,11141,1,1997-02-11,weekly,Hl.m. Praha,Prague,1204953,1,100.0,12541,...,0,0,1,0.2,0.43,85677.0,99107.0,1,1,1


In [143]:
combined4 = pd.merge(combined2, card_account, left_on='account_id', right_on='account_id', how='left')
combined4.rename(columns={'card_id': 'credit_cards'}, inplace=True)
combined4

,account_id,district_id,open_date,statement_frequency,district_name,region,population,num_cities,urban_ratio,avg_salary,...,crime_rate_96,id,num_customers,type,credit_cards,link_id,card_type,card_issue_date,client_id,card_type
0,1,18,1995-03-24,monthly,Pisek,south Bohemia,70699,4,65.3,8968,...,1910.0,1,1,1,0,0,0,0,1,1
1,2,1,1993-02-26,monthly,Hl.m. Praha,Prague,1204953,1,100.0,12541,...,99107.0,2,2,2,0,0,0,0,2,2
2,3,5,1997-07-07,monthly,Kolin,central Bohemia,95616,6,51.4,9307,...,3040.0,2,2,2,0,0,0,0,2,2
3,4,12,1996-02-21,monthly,Pribram,central Bohemia,107870,6,58.0,8754,...,3868.0,1,1,1,0,0,0,0,1,1
4,5,15,1997-05-30,monthly,Cesky Krumlov,south Bohemia,58796,5,51.9,9045,...,1879.0,1,1,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11027,32,1996-07-26,weekly,Ceska Lipa,north Bohemia,105058,10,81.0,9272,...,4505.0,1,1,1,0,0,0,0,1,1
4496,11054,59,1996-02-01,weekly,Kromeriz,south Moravia,108871,6,62.1,8444,...,2305.0,1,1,1,0,0,0,0,1,1
4497,11138,70,1995-08-18,weekly,Karvina,north Moravia,285387,7,89.9,10177,...,10108.0,1,1,1,1,1,1,1,1,1
4498,11141,1,1997-02-11,weekly,Hl.m. Praha,Prague,1204953,1,100.0,12541,...,99107.0,1,1,1,1,1,1,1,1,1


In [167]:
combined5 = pd.merge(combined4, loans3, left_on='account_id', right_on='account_id', how='left')
combined5.rename(columns={'id_y': 'loan'}, inplace=True)
combined5['loan'] = combined5['loan'].apply(lambda x: False if np.isnan(x) else True)
combined5.rename(columns={'amount': 'loan_amount', 'payments': 'loan_payments', 'term': 'loan_term', 'active_status': 'loan_status'}, inplace=True)
combined5['loan_default'] = combined5['payment'].apply(lambda x: True if x == 'default' else (False if x in ['full', 'in_progress'] else x))
combined5


,account_id,district_id,open_date,statement_frequency,district_name,region,population,num_cities,urban_ratio,avg_salary,...,date,loan_amount,loan_payments,variable,value,loan_term,status,loan_status,payment,loan_default
0,1,18,1995-03-24,monthly,Pisek,south Bohemia,70699,4,65.3,8968,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,1993-02-26,monthly,Hl.m. Praha,Prague,1204953,1,100.0,12541,...,1994-01-05,80952.0,3373.0,24_A,1.0,24.0,A,expired,full,False
2,3,5,1997-07-07,monthly,Kolin,central Bohemia,95616,6,51.4,9307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,12,1996-02-21,monthly,Pribram,central Bohemia,107870,6,58.0,8754,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,15,1997-05-30,monthly,Cesky Krumlov,south Bohemia,58796,5,51.9,9045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11027,32,1996-07-26,weekly,Ceska Lipa,north Bohemia,105058,10,81.0,9272,...,1997-10-23,48096.0,2004.0,24_C,1.0,24.0,C,current,in_progress,False
4496,11054,59,1996-02-01,weekly,Kromeriz,south Moravia,108871,6,62.1,8444,...,1996-08-20,148920.0,2482.0,60_C,1.0,60.0,C,current,in_progress,False
4497,11138,70,1995-08-18,weekly,Karvina,north Moravia,285387,7,89.9,10177,...,1996-01-19,89880.0,3745.0,24_A,1.0,24.0,A,expired,full,False
4498,11141,1,1997-02-11,weekly,Hl.m. Praha,Prague,1204953,1,100.0,12541,...,1997-10-07,44940.0,3745.0,12_A,1.0,12.0,A,expired,full,False


In [14]:
transactions = pd.read_csv('transactions.csv')
max_withdrawal = transactions.groupby(['account_id'])['amount'].agg(max)
min_withdrawal = transactions.groupby(['account_id'])['amount'].agg(min)

In [176]:
min_withdrawal

account_id
1        15
2        14
3        15
4        15
5         4
         ..
11333     0
11349    30
11359    15
11362    15
11382     1
Name: amount, Length: 4500, dtype: int64

In [177]:
combined6 = pd.merge(combined5, max_withdrawal, left_on='account_id', right_on='account_id', how='left')
combined7 = pd.merge(combined6, min_withdrawal, left_on='account_id', right_on='account_id', how='left')
combined7.rename(columns={'amount_x': 'max_withdrawal', 'amount_y': 'min_withdrawal'}, inplace=True)
combined7

,account_id,district_id,open_date,statement_frequency,district_name,region,population,num_cities,urban_ratio,avg_salary,...,loan_payments,variable,value,loan_term,status,loan_status,payment,loan_default,max_withdrawal,min_withdrawal
0,1,18,1995-03-24,monthly,Pisek,south Bohemia,70699,4,65.3,8968,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12600,15
1,2,1,1993-02-26,monthly,Hl.m. Praha,Prague,1204953,1,100.0,12541,...,3373.0,24_A,1.0,24.0,A,expired,full,False,42000,14
2,3,5,1997-07-07,monthly,Kolin,central Bohemia,95616,6,51.4,9307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11253,15
3,4,12,1996-02-21,monthly,Pribram,central Bohemia,107870,6,58.0,8754,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5553,15
4,5,15,1997-05-30,monthly,Cesky Krumlov,south Bohemia,58796,5,51.9,9045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5100,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11027,32,1996-07-26,weekly,Ceska Lipa,north Bohemia,105058,10,81.0,9272,...,2004.0,24_C,1.0,24.0,C,current,in_progress,False,20320,19
4496,11054,59,1996-02-01,weekly,Kromeriz,south Moravia,108871,6,62.1,8444,...,2482.0,60_C,1.0,60.0,C,current,in_progress,False,22780,30
4497,11138,70,1995-08-18,weekly,Karvina,north Moravia,285387,7,89.9,10177,...,3745.0,24_A,1.0,24.0,A,expired,full,False,64200,26
4498,11141,1,1997-02-11,weekly,Hl.m. Praha,Prague,1204953,1,100.0,12541,...,3745.0,12_A,1.0,12.0,A,expired,full,False,78600,30


In [184]:
payment_orders = pd.read_csv('payment_orders.csv')
payment_counts = payment_orders.groupby(['account_id'])['id'].count().reset_index()
payment_counts.columns = ['account_id', 'cc_payments']
combined8 = pd.merge(combined7, payment_counts, left_on='account_id', right_on='account_id',how='left')
combined8

,account_id,district_id,open_date,statement_frequency,district_name,region,population,num_cities,urban_ratio,avg_salary,...,variable,value,loan_term,status,loan_status,payment,loan_default,max_withdrawal,min_withdrawal,cc_payments
0,1,18,1995-03-24,monthly,Pisek,south Bohemia,70699,4,65.3,8968,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12600,15,1.0
1,2,1,1993-02-26,monthly,Hl.m. Praha,Prague,1204953,1,100.0,12541,...,24_A,1.0,24.0,A,expired,full,False,42000,14,2.0
2,3,5,1997-07-07,monthly,Kolin,central Bohemia,95616,6,51.4,9307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11253,15,3.0
3,4,12,1996-02-21,monthly,Pribram,central Bohemia,107870,6,58.0,8754,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5553,15,2.0
4,5,15,1997-05-30,monthly,Cesky Krumlov,south Bohemia,58796,5,51.9,9045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5100,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11027,32,1996-07-26,weekly,Ceska Lipa,north Bohemia,105058,10,81.0,9272,...,24_C,1.0,24.0,C,current,in_progress,False,20320,19,2.0
4496,11054,59,1996-02-01,weekly,Kromeriz,south Moravia,108871,6,62.1,8444,...,60_C,1.0,60.0,C,current,in_progress,False,22780,30,2.0
4497,11138,70,1995-08-18,weekly,Karvina,north Moravia,285387,7,89.9,10177,...,24_A,1.0,24.0,A,expired,full,False,64200,26,1.0
4498,11141,1,1997-02-11,weekly,Hl.m. Praha,Prague,1204953,1,100.0,12541,...,12_A,1.0,12.0,A,expired,full,False,78600,30,1.0


In [201]:
max_balance = transactions.groupby(['account_id'])['balance'].agg(max).to_frame().reset_index().rename(columns={'balance': 'max_balance'})
min_balance = transactions.groupby(['account_id'])['balance'].agg(min).to_frame().reset_index().rename(columns={'balance': 'min_balance'})
combined9 = pd.concat([combined8, max_balance, min_balance], axis=1)
combined9

,account_id,district_id,open_date,statement_frequency,district_name,region,population,num_cities,urban_ratio,avg_salary,...,loan_status,payment,loan_default,max_withdrawal,min_withdrawal,cc_payments,account_id,max_balance,account_id,min_balance
0,1,18,1995-03-24,monthly,Pisek,south Bohemia,70699,4,65.3,8968,...,NaN,NaN,NaN,12600,15,1.0,1,30415,1,1000
1,2,1,1993-02-26,monthly,Hl.m. Praha,Prague,1204953,1,100.0,12541,...,expired,full,False,42000,14,2.0,2,69302,2,1100
2,3,5,1997-07-07,monthly,Kolin,central Bohemia,95616,6,51.4,9307,...,NaN,NaN,NaN,11253,15,3.0,3,53447,3,1000
3,4,12,1996-02-21,monthly,Pribram,central Bohemia,107870,6,58.0,8754,...,NaN,NaN,NaN,5553,15,2.0,4,34870,4,800
4,5,15,1997-05-30,monthly,Cesky Krumlov,south Bohemia,58796,5,51.9,9045,...,NaN,NaN,NaN,5100,4,1.0,5,32036,5,600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11027,32,1996-07-26,weekly,Ceska Lipa,north Bohemia,105058,10,81.0,9272,...,current,in_progress,False,20320,19,2.0,11333,137043,11333,-1291
4496,11054,59,1996-02-01,weekly,Kromeriz,south Moravia,108871,6,62.1,8444,...,current,in_progress,False,22780,30,2.0,11349,113679,11349,200
4497,11138,70,1995-08-18,weekly,Karvina,north Moravia,285387,7,89.9,10177,...,expired,full,False,64200,26,1.0,11359,81706,11359,1000
4498,11141,1,1997-02-11,weekly,Hl.m. Praha,Prague,1204953,1,100.0,12541,...,expired,full,False,78600,30,1.0,11362,56056,11362,1000


In [203]:
final_df = combined9[['account_id', 'district_name', 'open_date', 'statement_frequency', 'num_customers', 'credit_cards', 'loan', 'loan_amount', 'loan_payments', 'loan_term', 'loan_status', 'loan_default', 'max_withdrawal', 'min_withdrawal', 'cc_payments', 'max_balance', 'min_balance']]
final_df = final_df.iloc[:,2:]
final_df

,account_id,district_name,open_date,statement_frequency,num_customers,credit_cards,loan,loan_amount,loan_payments,loan_term,loan_status,loan_default,max_withdrawal,min_withdrawal,cc_payments,max_balance,min_balance
0,1,Pisek,1995-03-24,monthly,1,0,False,NaN,NaN,NaN,NaN,NaN,12600,15,1.0,30415,1000
1,2,Hl.m. Praha,1993-02-26,monthly,2,0,True,80952.0,3373.0,24.0,expired,False,42000,14,2.0,69302,1100
2,3,Kolin,1997-07-07,monthly,2,0,False,NaN,NaN,NaN,NaN,NaN,11253,15,3.0,53447,1000
3,4,Pribram,1996-02-21,monthly,1,0,False,NaN,NaN,NaN,NaN,NaN,5553,15,2.0,34870,800
4,5,Cesky Krumlov,1997-05-30,monthly,1,0,False,NaN,NaN,NaN,NaN,NaN,5100,4,1.0,32036,600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11333,Ceska Lipa,1996-07-26,weekly,1,0,True,48096.0,2004.0,24.0,current,False,20320,19,2.0,137043,-1291
4496,11349,Kromeriz,1996-02-01,weekly,1,0,True,148920.0,2482.0,60.0,current,False,22780,30,2.0,113679,200
4497,11359,Karvina,1995-08-18,weekly,1,1,True,89880.0,3745.0,24.0,expired,False,64200,26,1.0,81706,1000
4498,11362,Hl.m. Praha,1997-02-11,weekly,1,1,True,44940.0,3745.0,12.0,expired,False,78600,30,1.0,56056,1000
